In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import xml.etree.ElementTree as ET
from pathlib import Path

In [6]:
# 공공 API 키 정보 로드

def load_key():
    api_key_file = open("./res/api_public_key.txt","r")
    key_data=api_key_file.readlines()
    my_key=key_data[0]
    api_key_file.close()
    
    return my_key

In [13]:
load_key()

'DuNAHD53YMihcAYk251OZibIu7IxjIwuoPP365VnntZyBrwmHQqRhdA3Ed%2BiLCH5oasIn9aZGKE7LcITX8mVeA%3D%3D'

In [30]:
#공공 API 요청

url = 'https://apis.data.go.kr/B552061/AccidentDeath/getRestTrafficAccidentDeath'

searchYear= 'searchYear=' + '2014'
siDo=  'siDo=' + '1100'
guGun = 'guGun='+'1116'
numOfRows = 'numOfRows=' + '10'
pageNo = 'pageNo=' + '1'

req_url = url + "?" + 'serviceKey=' + load_key() + "&" + searchYear + "&" + siDo + "&" + guGun + "&" +numOfRows + "&" + pageNo + "&"+ " type=xml"

response = requests.get(req_url)

In [32]:
#print(response.content)

In [51]:
def find_xml_items(response):
    root = ET.fromstring(response.content)
    item_list = []
    for child in root.find('body').find('items'):
        elements = child.findall('*')
        data = {}
        for element in elements:
            tag = element.tag.strip()
            text = element.text.strip()
            data[tag] = text
        item_list.append(data)  
    return item_list

In [52]:
items_list = find_xml_items(response)

In [53]:
result = pd.DataFrame(items_list)

In [54]:
result

,acc_year,occrrnc_dt,dght_cd,occrrnc_day_cd,dth_dnv_cnt,injpsn_cnt,se_dnv_cnt,sl_dnv_cnt,wnd_dnv_cnt,occrrnc_lc_sido_cd,...,acc_ty_cd,aslt_vtr_cd,road_frm_lclas_cd,road_frm_cd,wrngdo_isrty_vhcty_lclas_cd,dmge_isrty_vhcty_lclas_cd,occrrnc_lc_x_crd,occrrnc_lc_y_crd,lo_crd,la_crd
0,2014,2014030619,2,5,1,1,0,0,0,1100,...,22,04,01,05,01,05,957672,1945927,127.02104629,37.51165779
1,2014,2014031513,1,7,1,2,1,0,0,1100,...,01,05,01,05,01,12,959668,1945846,127.04363866,37.51101083
2,2014,2014032022,2,5,1,1,0,0,0,1100,...,02,05,01,05,01,12,959077,1945776,127.03695551,37.51035657
3,2014,2014033010,1,1,1,1,0,0,0,1100,...,22,01,01,05,01,01,965248,1941575,127.10697277,37.47274497
4,2014,2014042522,2,6,1,1,0,0,0,1100,...,01,05,01,Z1,01,12,958701,1943163,127.03284793,37.48678789
5,2014,2014051515,1,5,1,1,0,0,0,1100,...,05,05,01,05,03,12,958534,1945940,127.03079674,37.51180869
6,2014,2014052704,2,3,1,1,0,0,0,1100,...,01,05,01,05,01,12,958125,1944706,127.02624489,37.50066957
7,2014,2014060611,1,6,1,1,0,0,0,1100,...,05,07,01,Z1,01,12,961642,1943604,127.06609104,37.49088966
8,2014,2014061100,2,4,1,1,0,0,0,1100,...,01,01,01,05,01,12,961503,1942304,127.06458671,37.47916642
9,2014,2014061402,2,7,1,1,0,0,0,1100,...,05,03,02,08,01,12,960814,1946927,127.05654776,37.52080638


In [40]:
result.to_csv(searchYear + "_" + siDo +"_"+ guGun + '.csv', index=False, mode='a')

In [55]:
result = pd.read_csv(searchYear + "_" + siDo +"_"+ guGun + '.csv', index_col=None)

In [56]:
import pymysql

In [57]:
conn = pymysql.connect(host='192.168.0.103', port=3306 ,user='user1', password='u1234', db='project01', charset='utf8')

In [58]:
cur = conn.cursor()

In [59]:
for index, row in result.iterrows():
    cur.execute("""
            INSERT INTO traffic_accident_data_01 (
                acc_year,
                occrrnc_dt,
                dght_cd,
                occrrnc_day_cd,
                dth_dnv_cnt,
                injpsn_cnt,
                se_dnv_cnt,
                sl_dnv_cnt,
                wnd_dnv_cnt,
                occrrnc_lc_sido_cd,
                occrrnc_lc_sgg_cd,
                acc_ty_lclas_cd,
                acc_ty_mlsfc_cd,
                acc_ty_cd,
                aslt_vtr_cd,
                road_frm_lclas_cd,
                road_frm_cd,
                wrngdo_isrty_vhcty_lclas_cd,
                dmge_isrty_vhcty_lclas_cd,
                occrrnc_lc_x_crd,
                occrrnc_lc_y_crd,
                lo_crd,
                la_crd
            ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s,%s, %s, %s, %s, %s, %s, %s, %s)
        """, (
            row['acc_year'],
            row['occrrnc_dt'],
            row['dght_cd'],
            row['occrrnc_day_cd'],
            row['dth_dnv_cnt'],
            row['injpsn_cnt'],
            row['se_dnv_cnt'],
            row['sl_dnv_cnt'],
            row['wnd_dnv_cnt'],
            row['occrrnc_lc_sido_cd'],
            row['occrrnc_lc_sgg_cd'],
            row['acc_ty_lclas_cd'],
            row['acc_ty_mlsfc_cd'],
            row['acc_ty_cd'],
            row['aslt_vtr_cd'],
            row['road_frm_lclas_cd'],
            row['road_frm_cd'],
            row['wrngdo_isrty_vhcty_lclas_cd'],
            row['dmge_isrty_vhcty_lclas_cd'],
            row['occrrnc_lc_x_crd'],
            row['occrrnc_lc_y_crd'],
            row['lo_crd'],
            row['la_crd']
        ))

In [60]:
# 변경사항 커밋
conn.commit()
# 연결 종료
conn.close()